Generate EL points and simulate SiPM responses. 

Arrays (earrays) are created in tables and are extendable. They are stored in a group in f.root referring to the number of EL pts (NUM_ELPT) simulated per event in this toy montecarlo.

Because these data stored in different groups in f.root depending on NUM_ELPT, a single .h file created by this code could later be modified to contain new SiPM response data with different NUM_ELPT. New data for NUM_ELPT=2 (for example) in one group and old data for NUM_ELPT=1 in a different group.

with previous data under one f.root.group and new data under a new f.root.group. This means that if you run this code multiple times with different NUM_ELPT you can store data for different ELPTs/event in the same .h file. 

In [1]:
from __future__ import print_function


import random
import tables
import numpy as np

NSIPM = 8
NUM_ELPT = 1     # max num points observed in EL in same timestep 
                 
sipm_pitch = 10.0     # SiPM pitch in mm
sipm_edge_width = 5.0 # width of edge of dice board in mm
ze = 10.0             # distance between SiPM plane and EL gap
grid_space = 2.0      # grid spacing in mm
d_gap = 5.0           # length of EL gap
n_tbins = 2           # number of time bins collected as electron crosses EL

N = 1   # Number of photons (probably not necessary due to normalization)

nevts = 2 # num events for each num EL pt

max_xy = (NSIPM-1)*sipm_pitch + 2*sipm_edge_width # maximum x and y value (80 mm)
max_p = max_xy /grid_space                        # number of points per line (40)


# initialize
sipm_res = np.zeros((nevts,NSIPM**2),dtype=np.float64) #SiPM responses
x = np.empty((NUM_ELPT,nevts),dtype = np.float64)      #xcoord for evt
y = np.empty((NUM_ELPT,nevts),dtype = np.float64)      #ycoord for evt
E = np.empty((NUM_ELPT,nevts),dtype = np.float64)      #energy fraction for evt



Set up the SiPM positions

In [2]:
sipm_pos_x = []
for i in range(NSIPM): sipm_pos_x.extend([sipm_edge_width + i*sipm_pitch]*NSIPM)
sipm_pos_y = [sipm_edge_width + i*sipm_pitch for i in range(NSIPM)]*NSIPM

print(zip(sipm_pos_x,sipm_pos_y))


[(5.0, 5.0), (5.0, 15.0), (5.0, 25.0), (5.0, 35.0), (5.0, 45.0), (5.0, 55.0), (5.0, 65.0), (5.0, 75.0), (15.0, 5.0), (15.0, 15.0), (15.0, 25.0), (15.0, 35.0), (15.0, 45.0), (15.0, 55.0), (15.0, 65.0), (15.0, 75.0), (25.0, 5.0), (25.0, 15.0), (25.0, 25.0), (25.0, 35.0), (25.0, 45.0), (25.0, 55.0), (25.0, 65.0), (25.0, 75.0), (35.0, 5.0), (35.0, 15.0), (35.0, 25.0), (35.0, 35.0), (35.0, 45.0), (35.0, 55.0), (35.0, 65.0), (35.0, 75.0), (45.0, 5.0), (45.0, 15.0), (45.0, 25.0), (45.0, 35.0), (45.0, 45.0), (45.0, 55.0), (45.0, 65.0), (45.0, 75.0), (55.0, 5.0), (55.0, 15.0), (55.0, 25.0), (55.0, 35.0), (55.0, 45.0), (55.0, 55.0), (55.0, 65.0), (55.0, 75.0), (65.0, 5.0), (65.0, 15.0), (65.0, 25.0), (65.0, 35.0), (65.0, 45.0), (65.0, 55.0), (65.0, 65.0), (65.0, 75.0), (75.0, 5.0), (75.0, 15.0), (75.0, 25.0), (75.0, 35.0), (75.0, 45.0), (75.0, 55.0), (75.0, 65.0), (75.0, 75.0)]


Generate array of El events

In [3]:
#Generate energy fractions for each point from uniform distribution so that they sum to 1
E = np.random.uniform(0,1,(NUM_ELPT,nevts))
E = E / E.sum(axis=0)

#Generate x,y coords for each elpt
for i in range(NUM_ELPT):
    elpt = np.random.randint(0,1600,nevts)
    x[i] = (elpt % max_p)*grid_space + 1
    y[i] = (np.floor(elpt/max_p))*grid_space + 1
    
    




Define sipm response function

In [4]:
"""
sipm_param.py
author: jrenner
Defines the SiPM parameterization functions as:
N(x) = M*sum(c_n*x^n) for n = 0 to n = 9
where x is the distance of the SiPM from some central point of
light emission.
Because the response is characterized over several time bins, we
have several values for M and the coefficients.
"""
import numpy as np

# Number of time bins
n_tbins = 2

# Coefficients from S2 parameterization
M = [1.599, 1.599]
c0 = [7.72708346764e-05, 0.000116782596518]
c1 = [-1.69330613273e-07, 3.05115354927e-06]
c2 = [-1.52173658255e-06, -7.00800605142e-06]
c3 = [-2.4985972302e-07, 6.53907883449e-07]
c4 = [1.12327204397e-07, 8.95230202525e-08]
c5 = [-1.49353264606e-08, -2.27173290582e-08]
c6 = [1.04614146487e-09, 2.00740799864e-09]
c7 = [-4.19111362353e-11, -9.21915945523e-11]
c8 = [9.12129133361e-13, 2.20534216312e-12]
c9 = [-8.40089561697e-15, -2.1795164563e-14]

# Maximum radial extent of parameterization
rmax = 20.

# Return the SiPM response for the specified time bin and radial distance.
def sipm_par(tbin,r):

    # Ensure the time bin value is valid.
    if(tbin < 0 or tbin >= n_tbins):
        print("Invalid time bin in sipm_param: returning 0.0 ...")
        return 0.0

    # Calculate the response based on the parametrization.
    vpar = M[tbin]*(c0[tbin] + c1[tbin]*r + c2[tbin]*r**2 + c3[tbin]*r**3 + 
    c4[tbin]*r**4 + c5[tbin]*r**5 + c6[tbin]*r**6 + c7[tbin]*r**7 + 
    c8[tbin]*r**8 + c9[tbin]*r**9)

    # Zero the response for radii too large.
    if(hasattr(vpar, "__len__")):
        ret = np.zeros(len(vpar)); iret = 0
        for rv,pv in zip(r,vpar):
            if(rv < rmax):
                ret[iret] = pv
            iret += 1
        return ret
    else:
        if(r < rmax):
            return vpar
        return 0.0


Generate SiPM responses

In [5]:

posxy = zip(sipm_pos_x,sipm_pos_y)
#R = np.empty_like(sipm_res)


# generate responses of sipms 
p = 0
#for each EL point (NUM_ELPT)...
for e in E:
    xy = zip(x[p],y[p])
    
    evt = 0
    # for each event (xi,yi) are its coordinates... 
    for xi,yi in xy:
        idx = 0 
        
        # for each sipm get sipm_par.
        for posx,posy in posxy:
            
            #calculate distance between this sipm and xi,yi
            r = np.sqrt((posx - xi)**2 + (posy - yi)**2)
            
            #add this response to previous responses weighted by this event's energy fraction
            sipm_res[evt,idx] = sipm_res[evt,idx] +  sipm_par(0,r)*e[evt]
            if n_tbins == 2: 
                sipm_res[evt,idx] = sipm_res[evt,idx] +  sipm_par(1,r)*e[evt]
                
            idx += 1 #iterate thru SIPMs
        evt += 1 #iterate thru events
    p += 1 #iterate thru points
        
#multiply by number of photons, not neccessary when assuming infinite photons (set N = 1)
if n_tbins == 2: sipm_res = sipm_res * N / 2 #average time of 2 bins
else: sipm_res *= N

#Normalize for the DNN
mean = np.mean(sipm_res)
std = np.std(sipm_res)
sipm_res = (sipm_res - mean)/std


#print(sipm_res)
#print(x)

Put responses in a table

In [6]:
# Store "x" in a chunked array with level 5 BLOSC compression...
f = tables.open_file('resp.h', 'w')

filters = tables.Filters(complib='blosc', complevel=9, shuffle=False)
groupname = 'sim_' + str(NUM_ELPT) + 'pt'
group = f.create_group(f.root, groupname, 'Group for ' + str(NUM_ELPT) + ' ELPTs')

#x
atom = tables.Atom.from_dtype(x.dtype)
x1 = f.create_earray(group, 'x', atom, (0,nevts), filters=filters) 
                                                                         # extensible along first index for additional
                                                                         # x coords of additional
                                                                         # data points can be added (if multiple data                                                                        
#y                                                                       # points per event)
atom = tables.Atom.from_dtype(y.dtype)                                  
y1 = f.create_earray(group, 'y', atom, (0,nevts), filters=filters)


#sipm
atom = tables.Atom.from_dtype(sipm_res.dtype)  #sipm_res1 probably doesn't need to be extendable
sipm_res1 = f.create_earray(group, 'sipm_resp', atom, (0,nevts,NSIPM**2), filters=filters) 


for p in range(NUM_ELPT):
    x1.append([x[p]])  
    y1.append([y[p]])
sipm_res1.append([sipm_res])

print(f)

f.close()


resp.h (File) ''
Last modif.: 'Tue Aug 23 22:46:54 2016'
Object Tree: 
/ (RootGroup) ''
/sim_1pt (Group) 'Group for 1 ELPTs'
/sim_1pt/sipm_resp (EArray(1, 2, 64), shuffle, blosc(9)) ''
/sim_1pt/x (EArray(1, 2), shuffle, blosc(9)) ''
/sim_1pt/y (EArray(1, 2), shuffle, blosc(9)) ''



Done!